#### 导入工具包

In [ ]:
from imutils import contours
import numpy as np
import argparse
import cv2
import myutils

#### 创建一个“参数解析器”对象。


###### 像是初始化一个工具，这个工具专门用来定义你的程序需要哪些命令行参数，以及如何去解析这些参数。返回一个字典:{'image': 'my_card.png', 'template': 'ocr_a_font.png'}
###### 需要给定该python文件的脚本形参，推荐使用绝对路径
###### 在后面的代码里，你就可以通过 args["image"] 和 args["template"] 来轻松获取用户传入的文件路径了:

In [ ]:
# 设置参数
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-t", "--template", required=True,
	help="path to template OCR-A image")
args = vars(ap.parse_args())

##### 指定信用卡类型

In [ ]:
# 指定信用卡类型
FIRST_NUMBER = {
	"3": "American Express",
	"4": "Visa",
	"5": "MasterCard",
	"6": "Discover Card"
}

##### 创建绘图展示函数：展示银行卡图像

In [ ]:
# 绘图展示
def cv_show(name,img):
	cv2.imshow(name, img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

##### 读取一个模板图像

In [ ]:
img = cv2.imdecode(np.fromfile(args["template"], dtype=np.uint8), cv2.IMREAD_UNCHANGED)
cv_show('img',img)

##### 灰度图

In [ ]:
ref = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv_show('ref',ref)

 二值图像

In [ ]:
ref = cv2.threshold(ref, 10, 255, cv2.THRESH_BINARY_INV)[1]
cv_show('ref',ref)

##### 计算轮廓（得到图像轮廓信息）
###### cv2.findContours()函数接受的参数为二值图，即黑白的（不是灰度图）,cv2.RETR_EXTERNAL只检测外轮廓，cv2.CHAIN_APPROX_SIMPLE只保留终点坐标返回的list中每个元素都是图像中的一个轮廓

In [ ]:
refCnts, hierarchy = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#refCnts：得到图像的所有轮廓信息  ；hierarchy：一个包含轮廓层次结构信息的可选输出

###### 返回：一个根据图像二值图获得的红色轮廓图像
###### 重新将refCnts有顺序的进行排列
###### 创建一个空字典，方便后续存储着所有识别出的数字图像，以便地通过索引来取用每一个数字的图像进行下一步处理

In [ ]:
cv2.drawContours(img,refCnts,-1,(0,0,255),3)
cv_show('img',img)
#print (np.array(refCnts).shape)
refCnts = myutils.sort_contours(refCnts, method="left-to-right")[0]     #排序，从左到右，从上到下
digits = {}

遍历每一个轮廓，标准化图像尺寸，方便后续进行处理

In [ ]:
for (i, c) in enumerate(refCnts):
	# 计算外接矩形并且resize成合适大小
	(x, y, w, h) = cv2.boundingRect(c)
	roi = ref[y:y + h, x:x + w]
	roi = cv2.resize(roi, (57, 88))

	# 每一个数字对应每一个模板
	digits[i] = roi